# Quickstart for Carsus

In this guide you will learn how to create an atomic file suitable for [TARDIS](https://github.com/tardis-sn/tardis).

## Get familiar with Carsus notation

The first thing you should learn about Carsus is its notation: 

- **Carsus uses '0' for neutral elements.** `Si 0`  is equivalent to $\text{Si I}$, `Si 1` to $\text{Si II}$, etc.

- **Use a dash to grab intervals of consectutive elements or species.** `H-He` selects  $\text{H I}$ and $\text{H II}$ plus $\text{He I}$,  $\text{He II}$ and  $\text{He III}$, while `C 0-2` selects  $\text{C I}$,  $\text{C II}$ and $\text{C III}$. 

- **Use a comma to grab non-consecutive species.** `Si 0, 2` selects $\text{Si I}$ and $\text{Si III}$.
  
- **Use a semicolon to grab non-consecutive elements.** `H; Li` selects  $\text{H I}$ and $\text{H II}$ plus $\text{Li I}$,  $\text{Li II}$, $\text{Li III}$ and $\text{Li IV}$.

- **Finally, mix all the above syntax as needed.** For example, `H; C-Si; Fe 1,3`.

## Initialization and imports

In [1]:
from carsus.io.nist import NISTWeightsComp, NISTIonizationEnergies
from carsus.io.kurucz import GFALLReader
from carsus.io.zeta import KnoxLongZeta
from carsus.io.chianti_ import ChiantiReader
from carsus.io.output import TARDISAtomData

 ChiantiPy version 0.8.4 
 found PyQt5 widgets
 using PyQt5 widgets


## Atomic weights and ionization energies

Get atomic weights and ionization energies from NIST.

In [2]:
atomic_weights = NISTWeightsComp()

[carsus.io.nist.weightscomp][INFO] - Downloading data from the NIST Atomic Weights and Isotopic Compositions Database. (weightscomp.py:49)


In [3]:
ionization_energies = NISTIonizationEnergies('H-Zn')

[ carsus.io.nist.ionization][INFO] - Downloading ionization energies from the NIST Atomic Spectra Database. (ionization.py:65)


## Levels, lines and collisions

Currently, Carsus supports two sources of energy levels and transition lines: the Robert Kurucz's line list (GFALL) and the Chianti Database.

### GFALL


<div class="alert alert-warning">

**NOTE:** Creating a `GFALLReader` instance is **required**.

</div>

In [4]:
gfall_reader = GFALLReader('H-Zn')

[    carsus.io.kurucz.gfall][WARNING] - A specific combination to identify unique levels from GFALL data has not been given. Defaulting to ["energy", "j"]. (gfall.py:86)


### CHIANTI

The Chianti Database is also a source of collisional strengths.

In [5]:
chianti_reader = ChiantiReader('H-He', collisions=True, priority=20)

By default `priority` parameter is set to `10`. Increase this value if you want to keep Chianti levels and lines over GFALL.

## Zeta data

Adding Knox Long's ground state recombinations fractions ($\zeta$) is straightforward.

In [6]:
zeta_data = KnoxLongZeta()

## Save to HDF5

In [7]:
atom_data = TARDISAtomData(atomic_weights,
                           ionization_energies,
                           gfall_reader,
                           zeta_data,
                           chianti_reader)

[    carsus.io.kurucz.gfall][INFO] - Parsing GFALL: http://kurucz.harvard.edu/linelists/gfall/gfall.dat (gfall.py:139)


[     carsus.io.output.base][INFO] - Ingesting levels from Chianti. (base.py:255)


[     carsus.io.output.base][INFO] - Ingesting lines from GFALL. (base.py:327)


[    carsus.io.kurucz.gfall][INFO] - Extracting line data: atomic_number, ion_charge, energy_lower, j_lower, energy_upper, j_upper, wavelength, loggf. (gfall.py:343)


[               py.warnings][WARNING] - /usr/share/miniconda/envs/carsus/lib/python3.6/site-packages/carsus-0.1.dev637-py3.6.egg/carsus/io/output/base.py:121: PerformanceWarning: indexing past lexsort depth may impact performance.
  ['atomic_number', 'ion_number']).loc[ion]
 (warnings.py:99)


[     carsus.io.output.base][INFO] - Ingesting lines from Chianti. (base.py:352)


[               py.warnings][WARNING] - /usr/share/miniconda/envs/carsus/lib/python3.6/site-packages/carsus-0.1.dev637-py3.6.egg/carsus/io/output/base.py:356: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['line_id'] = range(start, len(df) + start)
 (warnings.py:99)


[     carsus.io.output.base][INFO] - Ingesting collisions from Chianti (base.py:499)


Finally, dump the `atom_data` attributes to an HDF5 file.

In [8]:
atom_data.to_hdf('kurucz_cd23_chianti_H-He.h5')

[     carsus.io.output.base][INFO] - Signing TARDISAtomData. (base.py:840)


[     carsus.io.output.base][INFO] - MD5: 6768f2d24d992a326ea0b8da57296bc5 (base.py:841)


[     carsus.io.output.base][INFO] - UUID1: 063c7a16ebb911ea9439000d3a8f33ce (base.py:842)


## Reproducibility and metadata

Carsus stores metadata inside the atomic files to ensure reproducibility. This metadata includes one checksum per `DataFrame` stored, versions of every dataset, and relevant package versions. 

In [9]:
import pandas as pd

In [10]:
pd.read_hdf('kurucz_cd23_chianti_H-He.h5', key='meta')

value
field    key                                              
md5sum   atom_data                    634785f05c4868f929c8
         collision_data               3178077be6f242541c21
         collision_data_temperatures  1b7075429ee24b944ed5
         ionization_data              160807a3951b9bdfddf9
         levels                       92f24fe12437a83c5b4b
         lines                        5c076a2fe6cb9f590542
         macro_atom_data              58a6eee3f36a8ba4ed69
         macro_atom_references        1138f02c1725cf9f2164
         zeta_data                    f2496cec5c219dfb6d63
datasets nist_weights                                  4.1
         nist_spectra                                5.7.1
         gfall.dat                    2704fbda0b8cba61bb70
         chianti_data                                9.0.1
software python                                     3.6.11
         carsus                                 0.1.dev637
         astropy                                     3.2.3
         numpy                                      1.15.4
         pandas                                      1.0.5
         pyarrow                                    0.14.1
         tables                                      3.6.1
         ChiantiPy                                   0.8.4

In [11]:
# Hidden cell
!rm kurucz_cd23_chianti_H-He.h5